> # **Using Images with CNN**


## 1) Installing Dependencies


In [ ]:
!pip install tensorflow --quiet

## 2) Importing Required Libraries


In [ ]:
import tensorflow as tf
from tensorflow import keras as ks
import os
import numpy as np

## 3) Loading the Image Data


In [ ]:
# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "/content/drive/MyDrive/mnist.npz")

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(
    path=data_path
)

print(f"Maximum pixel value before normalization: {np.max(training_images)}\n")
print(f"Shape of training set before reshaping: {training_images.shape}\n")
print(f"Shape of one image before reshaping: {training_images[0].shape}")

Maximum pixel value before normalization: 255

Shape of training set before reshaping: (60000, 28, 28)

Shape of one image before reshaping: (28, 28)


## 4) Pre-Processing the Data


In [ ]:
def reshape_and_normalize(images):
    # Reshape the images to add an extra dimension
    # this extra dimension represent the depth of color e.g. for
    # rgb it would be 3 but we have a grayscale so it is 1 which
    # is not neccessary to define but a good practice
    images = images.reshape(images.shape[0], 28, 28, 1)

    # Normalize pixel values
    images = images / 255.0

    return images

## 5) Reshaping the Data:


In [ ]:
# Reload the images in case you run this cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path)

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")

Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## 6) Defining Callback


In [ ]:
class myCallback(ks.callbacks.Callback):
    # defining a callback
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") is not None and logs.get("accuracy") > 0.995:
            print("\nReached 95% accuracy so cancelling training!")

            # Stop training once the above condition is met
            self.model.stop_training = True

## 5) Creating the Model


In [ ]:
def convolutional_model():
    # Define the Model
    model = ks.models.Sequential(
        [
            # defining convolutional network.
            # 64 is the filters: the dimension of the output space (the number of filters in the convolution)
            # 3x3 is the kernel size: the height and width of the 2D convolution window
            ks.layers.Conv2D(64, (3, 3), activation="relu", input_shape=(28, 28, 1)),
            # Downsamples the input along its spatial dimensions
            # (height and width) by taking the maximum value over an input window
            # (2, 2) is the poolsize: factors by which to downscale (dim1, dim2)
            ks.layers.MaxPooling2D(2, 2),
            ks.layers.Conv2D(64, (3, 3), activation="relu"),
            ks.layers.MaxPooling2D(2, 2),
            ks.layers.Flatten(),
            ks.layers.Dense(512, activation="relu"),
            ks.layers.Dense(10, activation="softmax"),
        ]
    )

    # Compile the model
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

    return model

## 6) Training the Model


In [ ]:
# Getting the model
model = convolutional_model()

# Get number of weights
model_params = model.count_params()

print(model_params)

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f"Your model has {model_params:,} params. For successful grading, please keep it "
    f"under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers."
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

862410
Epoch 1/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.1062 - accuracy: 0.9669
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0366 - accuracy: 0.9889
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0249 - accuracy: 0.9924
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0173 - accuracy: 0.9943
Epoch 5/10
1872/1875 [============================>.] - ETA: 0s - loss: 0.0116 - accuracy: 0.9963
Reached 95% accuracy so cancelling training!
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0116 - accuracy: 0.9962


## 7) Getting Model Summary


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 13, 13, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_4 (Dense)             (None, 512)              